In [1]:
from pathlib import Path
import tifffile
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import pyclesperanto_prototype as cle
from utils_cellpose import list_images, read_image, segment_nuclei_2d

cle.select_device("RTX")

c:\Users\adiez_cmic\miniforge3\envs\brain_nuc_cellpose\Lib\site-packages\cellpose\resnet_torch.py:276: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(

<NVIDIA GeForce RTX 4090 on Platform: NVIDIA CUDA (1 refs)>

<h3>Define the directory where your images are stored (.nd2 or .czi files)</h3>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("../raw_data/test_data")

# Construct ROI path from directory_path above
roi_path = directory_path / "ROIs"

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['..\\raw_data\\test_data\\HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi',
 '..\\raw_data\\test_data\\HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1.czi']

<h3>Define your nuclei channel and your nuclear segmentation parameters</h3>

Modify the values for <code>slicing factor</code>, <code>nuclei_channel</code>, <code>cellpose_nuclei_diameter</code> and <code>gaussian_sigma</code>


In [3]:
# Image size reduction to improve processing times (slicing, not lossless compression)
slicing_factor = None # Use 2 or 4 for compression (None for lossless)

# Define the nuclei and markers of interest channel order ('Remember in Python one starts counting from zero')
nuclei_channel = 3

# Segmentation type ("2D" or "3D"). 
# 2D takes a z-stack as input, performs MIP (Maximum Intensity Projection) and predicts nuclei from the resulting projection (faster, useful for single layers of cells)
# 3D is more computationally expensive. Predicts 3D nuclear volumes, useful for multilayered structures
segmentation_type = "2D"

# This is a placeholder to later choose from fine-tuned Cellpose models (default nuclei model in Cellpose 3.0)
model_name = "Cellpose"

# Define your nuclei diameter, it speeds up nuclei detection, if unknown leave it as None
cellpose_nuclei_diameter = None

# Define the amount of blur applied to nuclei
# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 0

<h3>Predict nuclei labels and store them as .tiff files</h3>

In [4]:
# List of subfolder names
try: 
    roi_names = [folder.name for folder in roi_path.iterdir() if folder.is_dir()]
    print(f"The following regions of interest will be analyzed: {roi_names}")

except FileNotFoundError:
    roi_names = ["full_image"]

for image in tqdm(images):

    # Generate maximum intensity projection and extract filename
    img_mip, filename = read_image(image, slicing_factor)

    # Slice the nuclei stack
    nuclei_img = img_mip[nuclei_channel, :, :]
    
    for roi_name in roi_names:
    
        print(f"Generating nuclei predictions for {roi_name} ROI")

        # Construct path to read ROI
        roi_path = directory_path / "ROIs" / roi_name / f"{filename}.tiff"

        try:

            # Read the .tiff files containing the user-defined ROIs
            roi = tifffile.imread(roi_path)

            # We will create a mask where roi is greater than or equal to 1
            mask = (roi >= 1).astype(np.uint8)

            # Apply the mask to nuclei_img and marker_img, setting all other pixels to 0
            masked_nuclei_img = np.where(mask, nuclei_img, 0)

        except FileNotFoundError:
            # If no ROI is saved the script will predict nuclei in the entire nuclei_img input
            masked_nuclei_img = nuclei_img

        # Segment nuclei and return labels
        nuclei_labels = segment_nuclei_2d(masked_nuclei_img, gaussian_sigma, cellpose_nuclei_diameter)

        # Save nuclei labels as .tiff files to reuse them later
        # Create nuclei_predictions directory if it does not exist
        try:
            os.makedirs(directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name)
        except FileExistsError:
            pass

        # Construct path to store
        nuclei_preds_path = directory_path / "nuclei_preds" / segmentation_type / model_name / roi_name / f"{filename}.tiff"

        # Save mask (binary image)
        tifffile.imwrite(nuclei_preds_path, nuclei_labels)

The following regions of interest will be analyzed: ['CA', 'DG']


  0%|          | 0/2 [00:00<?, ?it/s]

Image analyzed: HI 1  Contralateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 14, 3803, 2891)
MIP Array shape: (4, 3803, 2891)
Generating nuclei predictions for CA ROI
Generating nuclei predictions for DG ROI


 50%|█████     | 1/2 [00:23<00:23, 23.70s/it]

Image analyzed: HI 1  Ipsilateral Mouse 8  slide 6 Neun Red Calb Green KI67 Magenta 40x technical replica 1
Original Array shape: (4, 12, 3798, 2877)
MIP Array shape: (4, 3798, 2877)
Generating nuclei predictions for CA ROI
Generating nuclei predictions for DG ROI


100%|██████████| 2/2 [00:45<00:00, 22.78s/it]
